In [1]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply
import talib as ta
import importlib
import os
import platform
import psutil
import pandas as pd
import BacktestingStrategies.Optimization as opt

# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-5)
end = datetime.date.today()
#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html
# data = web.DataReader('7203.T', 'yahoo', start, end)
# data = web.DataReader('SPYD', 'yahoo', start, end)
# data = web.DataReader('1321.T', 'yahoo', start, end)
# data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
# data = web.DataReader('BTC-JPY', 'yahoo', start, end)
# data = web.DataReader('^N225', 'yahoo', start, end)
# data = web.DataReader('1655.T', 'yahoo', start, end)   #iシェアーズ S&P500 米国株 ETF　→　取得されるデータがおかしい？？
# data = web.DataReader('2558.T', 'yahoo', start, end)    #MAXIS 米国株式(S&P500)上場投信
# data = web.DataReader('2631.T', 'yahoo', start, end)    #MAXIS ナスダック100上場投信
# data = web.DataReader('2559.T', 'yahoo', start, end)      #MAXIS 全世界株式(オール・カントリー)上場投信
data = data.astype("double")
print(data)

initial = 10_000_000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

df_returns = pd.DataFrame()

              High     Low    Open   Close       Volume  Adj Close
Date                                                              
2017-09-21  1658.0  1633.0  1636.0  1653.0   13452172.0     1653.0
2017-09-22  1671.0  1644.0  1650.0  1662.0   13543147.0     1662.0
2017-09-25  1652.0  1638.0  1643.0  1647.0    8079821.0     1647.0
2017-09-26  1663.0  1649.0  1654.0  1661.0    9859990.0     1661.0
2017-09-27  1655.0  1644.0  1648.0  1644.0    7444797.0     1644.0
...            ...     ...     ...     ...          ...        ...
2022-09-13   346.0   342.0   345.0   344.0   46433034.0      344.0
2022-09-14   364.0   357.0   363.0   362.0  152405975.0      362.0
2022-09-15   364.0   359.0   361.0   361.0   40341207.0      361.0
2022-09-16   371.0   367.0   368.0   369.0   93123001.0      369.0
2022-09-20   368.0   360.0   362.0   365.0   57320464.0      365.0

[1236 rows x 6 columns]


In [2]:
def execute_backtest(bt):
    output = bt.run()
    print(output)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

In [3]:
def execute_optimize(bt, opt_params):
    output = bt.optimize(**opt_params)
    print(output)
    display(output._trades)
    display(output._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_temp["_strategy"] = str(output._strategy) + "_optimize"
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

**ボリンジャーバンドとRSI組み合わせ**

In [4]:
import BacktestingStrategies.Strategy_BBDandRSI as btstBBandRsi
importlib.reload(btstBBandRsi)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBandRsi.BBandRSI, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "upper_bound": range(50, 85, 5),
    "lower_bound": range(10, 45, 5),
    "rsi_window": range(10, 30, 2),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstBBandRsi.BBandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   47.087379
Equity Final [$]                8214811.80925
Equity Peak [$]                 16236815.4523
Return [%]                         -17.851882
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                   -3.929984
Volatility (Ann.) [%]               23.016954
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -59.203876
Avg. Drawdown [%]                   -7.715571
Max. Drawdown Duration     1111 days 00:00:00
Avg. Drawdown Duration      105 days 00:00:00
# Trades                                   14
Win Rate [%]                        57.142857
Best Trade [%]                      11.991766
Worst Trade [%]                    -31.980323
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7780,52,98,1285.33105,1389.0,8.065444e+05,0.080655,2017-12-04,2018-02-06,64 days
1,8961,167,180,1205.94000,1260.0,4.844317e+05,0.044828,2018-05-14,2018-05-31,17 days
2,10520,259,284,1073.28660,1191.0,1.238345e+06,0.109676,2018-09-19,2018-10-24,35 days
3,10984,313,318,1140.61825,1267.0,1.388177e+06,0.110801,2018-12-04,2018-12-11,7 days
4,11241,360,387,1238.09840,1212.0,-2.933721e+05,-0.021079,2019-02-14,2019-03-26,40 days
5,12506,402,416,1089.36580,1220.0,1.633711e+06,0.119918,2019-04-16,2019-05-14,28 days
6,13690,451,474,1114.48955,1186.0,9.789781e+05,0.064164,2019-07-02,2019-08-05,34 days
7,14147,497,592,1147.65290,910.0,-3.362076e+06,-0.207077,2019-09-06,2020-01-31,147 days
8,14171,668,714,908.47480,814.0,-1.338802e+06,-0.103993,2020-05-27,2020-08-03,68 days
9,15965,722,775,722.55905,714.0,-1.366452e+05,-0.011845,2020-08-14,2020-11-02,80 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   28.398058
Equity Final [$]                  8236652.607
Equity Peak [$]                11932327.17925
Return [%]                         -17.633474
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                   -3.877962
Volatility (Ann.) [%]               15.682532
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -39.214141
Avg. Drawdown [%]                   -8.321347
Max. Drawdown Duration     1160 days 00:00:00
Avg. Drawdown Duration      179 days 00:00:00
# Trades                                   22
Win Rate [%]                             50.0
Best Trade [%]                       7.540591
Worst Trade [%]                      -18.9878
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7184,27,40,1391.85575,1386.0,-4.206771e+04,-0.004207,2017-10-30,2017-11-16,17 days
1,8361,76,94,1190.86575,1227.0,3.021185e+05,0.030343,2018-01-05,2018-01-31,26 days
2,9559,259,274,1073.28660,1072.0,-1.229861e+04,-0.001199,2018-09-19,2018-10-10,21 days
3,8984,313,315,1140.61825,1213.0,6.502776e+05,0.063458,2018-12-04,2018-12-06,2 days
4,8802,360,377,1238.09840,1231.0,-6.248012e+04,-0.005733,2019-02-14,2019-03-11,25 days
5,9946,402,412,1089.36580,1134.0,4.439318e+05,0.040973,2019-04-16,2019-05-08,22 days
6,10120,451,463,1114.48955,1163.0,4.909258e+05,0.043527,2019-07-02,2019-07-19,17 days
7,10256,497,515,1147.65290,1104.0,-4.477041e+05,-0.038037,2019-09-06,2019-10-04,28 days
8,11334,523,556,998.92030,914.0,-9.624867e+05,-0.085012,2019-10-17,2019-12-05,49 days
9,11918,563,575,869.28175,906.0,4.376081e+05,0.042240,2019-12-16,2020-01-07,22 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7780,52,98,1285.33105,1389.0,8.065444e+05,0.080655,2017-12-04,2018-02-06,64 days
1,-8283,99,100,1304.51055,1382.0,-6.418451e+05,-0.059401,2018-02-07,2018-02-08,1 days
2,-7020,101,102,1447.79775,1422.0,1.811002e+05,0.017819,2018-02-09,2018-02-12,3 days
3,-7507,103,104,1378.14425,1439.0,-4.568441e+05,-0.044158,2018-02-13,2018-02-14,1 days
4,-6482,132,180,1525.41165,1260.0,1.720398e+06,0.173993,2018-03-26,2018-05-31,66 days
5,-9107,203,204,1274.65905,1311.0,-3.309570e+05,-0.028510,2018-07-03,2018-07-04,1 days
6,-8678,205,206,1299.53530,1308.0,-7.345667e+04,-0.006514,2018-07-05,2018-07-06,1 days
7,-8994,233,284,1245.80260,1191.0,4.928946e+05,0.043990,2018-08-14,2018-10-24,71 days
8,-9285,286,287,1259.73330,1283.0,-2.160313e+05,-0.018470,2018-10-26,2018-10-29,3 days
9,-8721,288,318,1316.45115,1267.0,4.312635e+05,0.037564,2018-10-30,2018-12-11,42 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   68.446602
Equity Final [$]                 13271925.331
Equity Peak [$]                 17507175.3626
Return [%]                          32.719253
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                    5.941032
Volatility (Ann.) [%]               31.217804
Sharpe Ratio                         0.190309
Sortino Ratio                        0.280002
Calmar Ratio                         0.104808
Max. Drawdown [%]                  -56.684828
Avg. Drawdown [%]                   -6.356178
Max. Drawdown Duration      533 days 00:00:00
Avg. Drawdown Duration       62 days 00:00:00
# Trades                                   86
Win Rate [%]                        51.162791
Best Trade [%]                      38.190282
Worst Trade [%]                     -21.10444
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7184,27,40,1391.85575,1386.0,-42067.70800,-0.004207,2017-10-30,2017-11-16,17 days
1,-7479,55,62,1331.37690,1285.0,346852.83510,0.034834,2017-12-07,2017-12-18,11 days
2,8653,76,94,1190.86575,1227.0,312669.66525,0.030343,2018-01-05,2018-01-31,26 days
3,-7681,98,99,1382.12445,1311.0,546306.90045,0.051460,2018-02-06,2018-02-07,1 days
4,-8118,100,101,1375.15910,1455.0,-648148.42620,-0.058059,2018-02-08,2018-02-09,1 days
...,...,...,...,...,...,...,...,...,...,...
81,-29190,1146,1151,433.84180,418.0,462422.14200,0.036515,2022-05-13,2022-05-20,7 days
82,34014,1162,1168,385.90080,415.0,989780.18880,0.075406,2022-06-06,2022-06-14,8 days
83,-34349,1181,1189,410.95565,412.0,-35872.37815,-0.002541,2022-07-01,2022-07-13,12 days
84,37362,1194,1221,376.85625,358.0,-704507.21250,-0.050036,2022-07-21,2022-08-30,40 days


<Strategy BBandRSI_WithShortPosition(upper_bound=50,lower_bound=10,rsi_window=10)>

**ボリンジャーバンドとRSI組み合わせ EntryRSI, ExitBolinger**

In [5]:
import BacktestingStrategies.Strategy_BBDandRSI as btstBBandRsi
importlib.reload(btstBBandRsi)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBandRsi.EntryRSI50andExitBB, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstBBandRsi.EntryRSI50andExitBB_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   32.443366
Equity Final [$]                 7332736.7276
Equity Peak [$]                  12690963.803
Return [%]                         -26.672633
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                   -6.129317
Volatility (Ann.) [%]               21.231635
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -51.163213
Avg. Drawdown [%]                  -14.860971
Max. Drawdown Duration     1541 days 00:00:00
Avg. Drawdown Duration      332 days 00:00:00
# Trades                                   46
Win Rate [%]                        47.826087
Best Trade [%]                      13.332023
Worst Trade [%]                    -12.308543
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7437,55,57,1344.62310,1272.0,-5.400980e+05,-0.054010,2017-12-07,2017-12-11,4 days
1,7671,94,98,1233.07365,1389.0,1.196111e+06,0.126453,2018-01-31,2018-02-06,6 days
2,7877,115,132,1352.66270,1533.0,1.420517e+06,0.133320,2018-03-01,2018-03-26,25 days
3,9683,176,180,1247.14295,1260.0,1.244948e+05,0.010309,2018-05-25,2018-05-31,6 days
4,9822,194,203,1242.11820,1281.0,3.818970e+05,0.031303,2018-06-20,2018-07-03,13 days
5,9921,210,214,1268.24690,1165.0,-1.024312e+06,-0.081409,2018-07-12,2018-07-18,6 days
6,9230,232,233,1252.16770,1252.0,-1.547871e+03,-0.000134,2018-08-13,2018-08-14,1 days
7,9488,251,257,1217.99940,1133.0,-8.064743e+05,-0.069786,2018-09-07,2018-09-17,10 days
8,9166,276,284,1172.77665,1191.0,1.670352e+05,0.015539,2018-10-12,2018-10-24,12 days
9,9098,293,313,1199.91030,1135.0,-5.905539e+05,-0.054096,2018-11-06,2018-12-04,28 days


Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   35.194175
Equity Final [$]                6026984.15305
Equity Peak [$]                 12087051.0991
Return [%]                         -39.730158
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                   -9.808413
Volatility (Ann.) [%]               20.161484
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -56.392829
Avg. Drawdown [%]                  -13.963821
Max. Drawdown Duration     1499 days 00:00:00
Avg. Drawdown Duration      287 days 00:00:00
# Trades                                   72
Win Rate [%]                        44.444444
Best Trade [%]                      13.332023
Worst Trade [%]                     -22.26449
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7437,55,57,1344.62310,1272.0,-5.400980e+05,-0.054010,2017-12-07,2017-12-11,4 days
1,7671,94,98,1233.07365,1389.0,1.196111e+06,0.126453,2018-01-31,2018-02-06,6 days
2,-8308,113,114,1282.61945,1303.0,-1.693216e+05,-0.015890,2018-02-27,2018-02-28,1 days
3,7752,115,132,1352.66270,1533.0,1.397975e+06,0.133320,2018-03-01,2018-03-26,25 days
4,-8775,137,138,1354.26305,1408.0,-4.715417e+05,-0.039680,2018-04-02,2018-04-03,1 days
...,...,...,...,...,...,...,...,...,...,...
67,-17304,1148,1149,413.94080,407.0,1.201036e+05,0.016768,2022-05-17,2022-05-18,1 days
68,17337,1151,1162,420.06910,384.0,-6.253300e+05,-0.085865,2022-05-20,2022-06-06,17 days
69,15963,1168,1194,417.05425,375.0,-6.713120e+05,-0.100836,2022-06-14,2022-07-21,37 days
70,16639,1221,1232,359.77210,363.0,5.370903e+04,0.008972,2022-08-30,2022-09-14,15 days


**MACDとRSIの組み合わせ**

In [6]:
import BacktestingStrategies.Strategy_MACDandRSI as btstMACDandRsi
importlib.reload(btstMACDandRsi)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACDandRsi.MACDandRSI, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "MACDshort": range(10, 25, 5),\
    "MACDlong": range(10, 75, 10),\
    "MACDsignal": range(10, 15, 5),\
    "upper_bound": range(50, 85, 5),\
    "lower_bound": range(10, 45, 5),\
    "rsi_window": range(10, 30, 2),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "constraint": lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    "max_tries": 1000}
execute_optimize(bt, opt_params)


if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACDandRsi.MACDandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                    1.860841
Equity Final [$]                 9396731.9854
Equity Peak [$]                 10189730.9854
Return [%]                           -6.03268
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                   -1.260614
Volatility (Ann.) [%]                3.301955
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -8.528586
Avg. Drawdown [%]                   -5.338382
Max. Drawdown Duration     1034 days 00:00:00
Avg. Drawdown Duration      520 days 00:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                      -6.032823
Worst Trade [%]                     -6.032823
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,10863,541,563,920.5342,865.0,-603268.0146,-0.060328,2019-11-14,2019-12-16,32 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 519 configurations.
  output = _optimize_grid()


Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                    5.097087
Equity Final [$]                8145520.87975
Equity Peak [$]                10210061.07935
Return [%]                         -18.544791
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                   -4.095756
Volatility (Ann.) [%]                5.675858
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -20.220645
Avg. Drawdown [%]                  -10.541817
Max. Drawdown Duration     1770 days 00:00:00
Avg. Drawdown Duration      886 days 00:00:00
# Trades                                    2
Win Rate [%]                              0.0
Best Trade [%]                      -7.227781
Worst Trade [%]                    -12.199319
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7409,38,76,1349.64785,1185.0,-1.219876e+06,-0.121993,2017-11-14,2018-01-05,52 days
1,9538,541,564,920.53420,854.0,-6.346032e+05,-0.072278,2019-11-14,2019-12-17,33 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-8480,76,89,1179.13425,1146.0,2.809784e+05,0.028100,2018-01-05,2018-01-24,19 days
1,-7881,99,100,1304.51055,1382.0,-6.106944e+05,-0.059401,2018-02-07,2018-02-08,1 days
2,-6834,102,103,1414.96110,1385.0,2.047542e+05,0.021175,2018-02-12,2018-02-13,1 days
3,-6896,104,105,1431.87695,1420.0,8.190345e+04,0.008295,2018-02-14,2018-02-15,1 days
4,-7314,110,123,1361.22840,1359.0,1.629852e+04,0.001637,2018-02-22,2018-03-13,19 days
5,-7332,136,168,1360.23335,1187.0,1.270147e+06,0.127356,2018-03-30,2018-05-15,46 days
6,-9551,171,187,1177.14415,1203.0,-2.469492e+05,-0.021965,2018-05-18,2018-06-11,24 days
7,-9103,211,242,1207.99070,1181.0,2.456963e+05,0.022343,2018-07-13,2018-08-27,45 days
8,-9875,256,286,1138.33720,1266.0,-1.260670e+06,-0.112148,2018-09-14,2018-10-26,42 days
9,-7818,287,288,1276.64915,1323.0,-3.623709e+05,-0.036307,2018-10-29,2018-10-30,1 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 521 configurations.
  output = _optimize_grid()


Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   55.016181
Equity Final [$]                 2687481.2664
Equity Peak [$]                 11126037.4174
Return [%]                         -73.125187
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                  -23.501494
Volatility (Ann.) [%]               21.315542
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -75.845117
Avg. Drawdown [%]                  -11.738291
Max. Drawdown Duration     1691 days 00:00:00
Avg. Drawdown Duration      246 days 00:00:00
# Trades                                  211
Win Rate [%]                        44.549763
Best Trade [%]                      12.779157
Worst Trade [%]                     -21.10444
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-7511,55,76,1331.37690,1185.0,1.099437e+06,0.109944,2017-12-07,2018-01-05,29 days
1,-9090,94,95,1220.92635,1218.0,2.660052e+04,0.002397,2018-01-31,2018-02-01,1 days
2,-8853,97,98,1256.74815,1389.0,-1.170826e+06,-0.105233,2018-02-05,2018-02-06,1 days
3,-7631,99,100,1304.51055,1382.0,-5.913220e+05,-0.059401,2018-02-07,2018-02-08,1 days
4,-6467,101,102,1447.79775,1422.0,1.668340e+05,0.017819,2018-02-09,2018-02-12,3 days
...,...,...,...,...,...,...,...,...,...,...
206,-7920,1221,1224,356.22790,366.0,-7.739503e+04,-0.027432,2022-08-30,2022-09-02,3 days
207,-7433,1225,1226,369.16355,369.0,1.215667e+03,0.000443,2022-09-05,2022-09-06,1 days
208,-7416,1227,1228,370.15860,367.0,2.342418e+04,0.008533,2022-09-07,2022-09-08,1 days
209,-7707,1233,1234,359.21305,368.0,-6.772102e+04,-0.024462,2022-09-15,2022-09-16,1 days


<Strategy MACDandRSI_WithShortPosition(MACDshort=15,MACDlong=20,MACDsignal=10,upper_bound=50,lower_bound=10,rsi_window=14)>

**MACDとボリンジャーバンドの組み合わせ**

In [7]:
import BacktestingStrategies.Strategy_MACDandBBD as btstMACDandBB
importlib.reload(btstMACDandBB)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACDandBB.MACDandBBD, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)


# #最適化
opt_params = {
    "MACDshort": range(10, 25, 5),\
    "MACDlong": range(10, 75, 10),\
    "MACDsignal": range(10, 15, 5),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "constraint": lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACDandBB.MACDandBBD_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   20.307443
Equity Final [$]                4648365.13475
Equity Peak [$]                 12097019.9526
Return [%]                         -53.516349
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                  -14.460244
Volatility (Ann.) [%]               20.223624
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -63.549327
Avg. Drawdown [%]                  -25.539254
Max. Drawdown Duration      915 days 00:00:00
Avg. Drawdown Duration      420 days 00:00:00
# Trades                                   23
Win Rate [%]                        13.043478
Best Trade [%]                      27.233969
Worst Trade [%]                     -12.34401
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7163,98,110,1395.87555,1368.0,-1.996726e+05,-0.019970,2018-02-06,2018-02-22,16 days
1,6361,132,136,1540.58835,1367.0,-1.104195e+06,-0.112677,2018-03-26,2018-03-30,4 days
2,6867,180,187,1266.23700,1203.0,-4.342485e+05,-0.049941,2018-05-31,2018-06-11,11 days
3,6417,203,211,1287.34095,1214.0,-4.706289e+05,-0.056971,2018-07-03,2018-07-13,10 days
4,6192,233,242,1258.19740,1181.0,-4.780063e+05,-0.061356,2018-08-14,2018-08-27,13 days
5,6110,284,293,1196.89545,1194.0,-1.769120e+04,-0.002419,2018-10-24,2018-11-06,13 days
6,5729,318,336,1273.27165,1340.0,3.822867e+05,0.052407,2018-12-11,2019-01-09,29 days
7,6303,387,393,1217.99940,1145.0,-4.601152e+05,-0.059934,2019-03-26,2019-04-03,8 days
8,5887,416,436,1226.03900,1184.0,-2.474836e+05,-0.034288,2019-05-14,2019-06-11,28 days
9,5848,474,488,1191.87070,1262.0,4.101161e+05,0.058840,2019-08-05,2019-08-26,21 days


Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                    20.79288
Equity Final [$]                4059755.71575
Equity Peak [$]                11646908.97105
Return [%]                         -59.402443
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                   -16.78922
Volatility (Ann.) [%]               19.756833
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -65.143063
Avg. Drawdown [%]                  -26.548369
Max. Drawdown Duration      915 days 00:00:00
Avg. Drawdown Duration      420 days 00:00:00
# Trades                                   23
Win Rate [%]                        13.043478
Best Trade [%]                      27.233969
Worst Trade [%]                     -12.34401
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7163,98,110,1395.87555,1368.0,-1.996726e+05,-0.019970,2018-02-06,2018-02-22,16 days
1,6361,132,136,1540.58835,1367.0,-1.104195e+06,-0.112677,2018-03-26,2018-03-30,4 days
2,6867,180,188,1266.23700,1172.0,-6.471255e+05,-0.074423,2018-05-31,2018-06-12,12 days
3,6252,203,211,1287.34095,1214.0,-4.585276e+05,-0.056971,2018-07-03,2018-07-13,10 days
4,6032,233,242,1258.19740,1181.0,-4.656547e+05,-0.061356,2018-08-14,2018-08-27,13 days
5,5952,284,294,1196.89545,1180.0,-1.005617e+05,-0.014116,2018-10-24,2018-11-07,14 days
6,5516,318,336,1273.27165,1340.0,3.680736e+05,0.052407,2018-12-11,2019-01-09,29 days
7,6069,387,393,1217.99940,1145.0,-4.430334e+05,-0.059934,2019-03-26,2019-04-03,8 days
8,5668,416,436,1226.03900,1184.0,-2.382771e+05,-0.034288,2019-05-14,2019-06-11,28 days
9,5630,474,488,1191.87070,1262.0,3.948280e+05,0.058840,2019-08-05,2019-08-26,21 days


<Strategy MACDandBBD(MACDshort=15,MACDlong=20,MACDsignal=10)>

Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   40.372168
Equity Final [$]                 8087130.1161
Equity Peak [$]                 11743450.4651
Return [%]                         -19.128699
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                   -4.236325
Volatility (Ann.) [%]               28.792352
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -50.226364
Avg. Drawdown [%]                  -13.088902
Max. Drawdown Duration     1064 days 00:00:00
Avg. Drawdown Duration      186 days 00:00:00
# Trades                                   31
Win Rate [%]                        32.258065
Best Trade [%]                      27.233969
Worst Trade [%]                    -14.672511
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-8480,76,89,1179.13425,1146.0,2.809784e+05,0.028100,2018-01-05,2018-01-24,19 days
1,7365,98,110,1395.87555,1368.0,-2.053034e+05,-0.019970,2018-02-06,2018-02-22,16 days
2,-7450,123,136,1352.27295,1367.0,-1.097165e+05,-0.010891,2018-03-13,2018-03-30,17 days
3,-8437,168,171,1181.12435,1183.0,-1.582486e+04,-0.001588,2018-05-15,2018-05-18,3 days
4,7858,180,187,1266.23700,1203.0,-4.969163e+05,-0.049941,2018-05-31,2018-06-11,11 days
5,7343,203,211,1287.34095,1214.0,-5.385426e+05,-0.056971,2018-07-03,2018-07-13,10 days
6,7085,233,242,1258.19740,1181.0,-5.469436e+05,-0.061356,2018-08-14,2018-08-27,13 days
7,-7350,256,293,1138.33720,1194.0,-4.091216e+05,-0.048898,2018-09-14,2018-11-06,53 days
8,-6693,308,336,1189.08475,1340.0,-1.010076e+06,-0.126917,2018-11-27,2019-01-09,43 days
9,5704,387,393,1217.99940,1145.0,-4.163886e+05,-0.059934,2019-03-26,2019-04-03,8 days


Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   37.055016
Equity Final [$]                5099016.24275
Equity Peak [$]                 11629278.3856
Return [%]                         -49.009838
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                  -12.831153
Volatility (Ann.) [%]               26.070927
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -59.609186
Avg. Drawdown [%]                  -22.860087
Max. Drawdown Duration      915 days 00:00:00
Avg. Drawdown Duration      343 days 00:00:00
# Trades                                   32
Win Rate [%]                           34.375
Best Trade [%]                      27.233969
Worst Trade [%]                    -16.017324
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-8480,76,110,1179.13425,1368.0,-1.601582e+06,-0.160173,2018-01-05,2018-02-22,48 days
1,-6317,122,136,1329.38680,1367.0,-2.376026e+05,-0.028294,2018-03-12,2018-03-30,18 days
2,-6909,168,173,1181.12435,1174.0,4.922213e+04,0.006032,2018-05-15,2018-05-22,7 days
3,6483,180,188,1266.23700,1172.0,-6.109385e+05,-0.074423,2018-05-31,2018-06-12,12 days
4,5902,203,211,1287.34095,1214.0,-4.328583e+05,-0.056971,2018-07-03,2018-07-13,10 days
5,-6021,230,242,1190.07980,1181.0,5.466948e+04,0.007630,2018-08-09,2018-08-27,18 days
6,-6343,256,294,1138.33720,1180.0,-2.642671e+05,-0.036600,2018-09-14,2018-11-07,54 days
7,-5683,307,336,1223.91150,1340.0,-6.597309e+05,-0.094850,2018-11-26,2019-01-09,44 days
8,5169,387,393,1217.99940,1145.0,-3.773339e+05,-0.059934,2019-03-26,2019-04-03,8 days
9,4828,416,436,1226.03900,1184.0,-2.029643e+05,-0.034288,2019-05-14,2019-06-11,28 days


<Strategy MACDandBBD_WithShortPosition(MACDshort=15,MACDlong=20,MACDsignal=10)>

SMA and RSI

In [8]:
import BacktestingStrategies.Strategy_SMAandRSI as btstSMAandRsi
importlib.reload(btstSMAandRsi)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstSMAandRsi.EntryRSIandExitSMA_WithShortPosition, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)


# #最適化
opt_params = {
    "SMA_short": range(10, 25, 5),\
    "SMA_long": range(10, 75, 10),\
    "upper_bound": range(50, 85, 5),\
    "lower_bound": range(10, 45, 5),\
    "rsi_window": range(10, 30, 2),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "constraint": lambda x: (x.SMA_short < x.SMA_long),\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstSMAandRsi.SMAandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   39.967638
Equity Final [$]                10508894.8168
Equity Peak [$]                 15415839.9964
Return [%]                           5.088948
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                    1.017153
Volatility (Ann.) [%]               42.682151
Sharpe Ratio                         0.023831
Sortino Ratio                        0.044219
Calmar Ratio                         0.015574
Max. Drawdown [%]                  -65.309396
Avg. Drawdown [%]                   -7.995797
Max. Drawdown Duration     1083 days 00:00:00
Avg. Drawdown Duration       98 days 00:00:00
# Trades                                   30
Win Rate [%]                        33.333333
Best Trade [%]                      20.607183
Worst Trade [%]                    -11.081161
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7515,31,35,1330.55380,1257.80,-5.467448e+05,-0.054679,2017-11-03,2017-11-09,6 days
1,7169,36,75,1318.49440,1217.90,-7.211613e+05,-0.076295,2017-11-10,2018-01-04,55 days
2,7332,76,85,1190.86575,1136.00,-4.022757e+05,-0.046072,2018-01-05,2018-01-18,13 days
3,7232,89,98,1151.67270,1389.00,1.716351e+06,0.206072,2018-01-24,2018-02-06,13 days
4,-7305,100,128,1375.15910,1417.00,-3.056478e+05,-0.030426,2018-02-08,2018-03-20,40 days
5,8077,167,182,1205.94000,1238.00,2.589486e+05,0.026585,2018-05-14,2018-06-04,21 days
6,9316,259,264,1073.28660,1045.00,-2.635180e+05,-0.026355,2018-09-19,2018-09-26,7 days
7,9324,265,277,1044.14305,1185.65,1.319411e+06,0.135524,2018-09-27,2018-10-15,18 days
8,-8775,286,312,1259.73330,1125.00,1.182285e+06,0.106954,2018-10-26,2018-12-03,38 days
9,-8171,329,345,1497.55025,1324.00,1.418079e+06,0.115889,2018-12-26,2019-01-23,28 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 834 configurations.
  output = _optimize_grid()


Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   74.595469
Equity Final [$]                10213999.5959
Equity Peak [$]                 11791760.3035
Return [%]                           2.139996
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                     0.43264
Volatility (Ann.) [%]               44.495565
Sharpe Ratio                         0.009723
Sortino Ratio                        0.015996
Calmar Ratio                         0.006477
Max. Drawdown [%]                  -66.795929
Avg. Drawdown [%]                   -18.45876
Max. Drawdown Duration     1449 days 00:00:00
Avg. Drawdown Duration      355 days 00:00:00
# Trades                                   66
Win Rate [%]                        53.030303
Best Trade [%]                       39.82143
Worst Trade [%]                    -18.616266
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,6737,21,26,1484.31115,1393.65,-610784.16755,-0.061080,2017-10-20,2017-10-27,7 days
1,6745,27,32,1391.85575,1313.00,-531882.03375,-0.056655,2017-10-30,2017-11-06,7 days
2,6656,33,35,1330.55380,1255.90,-496895.69280,-0.056107,2017-11-07,2017-11-09,2 days
3,6340,36,46,1318.49440,1332.00,85625.50400,0.010243,2017-11-10,2017-11-24,14 days
4,-6343,55,95,1331.37690,1218.00,719149.67670,0.085158,2017-12-07,2018-02-01,56 days
...,...,...,...,...,...,...,...,...,...,...
61,-22640,1137,1155,408.96555,410.00,-23419.94800,-0.002529,2022-04-26,2022-05-26,30 days
62,-22365,1168,1173,412.94575,428.00,-336688.30125,-0.036456,2022-06-14,2022-06-21,7 days
63,-21602,1174,1186,411.95070,405.00,150149.02140,0.016873,2022-06-22,2022-07-08,16 days
64,-22073,1189,1224,409.96060,366.00,970342.32380,0.107231,2022-07-13,2022-09-02,51 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-7590,31,32,1317.44620,1313.0,33746.65800,0.003375,2017-11-03,2017-11-06,3 days
1,-7616,33,34,1317.44620,1287.0,231878.25920,0.023110,2017-11-07,2017-11-08,1 days
2,-8136,35,36,1261.72340,1312.0,-409050.41760,-0.039848,2017-11-09,2017-11-10,1 days
3,-7550,37,38,1305.50560,1343.0,-283082.72000,-0.028720,2017-11-13,2017-11-14,1 days
4,-7110,39,40,1346.30265,1386.0,-282248.15850,-0.029486,2017-11-15,2017-11-16,1 days
...,...,...,...,...,...,...,...,...,...,...
406,-8861,1216,1217,345.28235,350.0,-41803.09665,-0.013663,2022-08-23,2022-08-24,1 days
407,-8640,1218,1219,349.26255,344.0,45468.43200,0.015068,2022-08-25,2022-08-26,1 days
408,-8481,1220,1221,361.20315,358.0,27165.91515,0.008868,2022-08-29,2022-08-30,1 days
409,-8580,1222,1223,360.20810,365.0,-41114.50200,-0.013303,2022-08-31,2022-09-01,1 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 811 configurations.
  output = _optimize_grid()


Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                    91.18123
Equity Final [$]                 500568.39605
Equity Peak [$]                 10378079.7568
Return [%]                         -94.994316
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                   -45.69472
Volatility (Ann.) [%]               15.368389
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -95.270566
Avg. Drawdown [%]                  -32.759877
Max. Drawdown Duration     1764 days 00:00:00
Avg. Drawdown Duration      595 days 00:00:00
# Trades                                  513
Win Rate [%]                        39.766082
Best Trade [%]                      14.800104
Worst Trade [%]                    -13.766076
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-7567,30,31,1321.42640,1324.0,-19474.43120,-0.001948,2017-11-02,2017-11-03,1 days
1,-7639,32,33,1306.50065,1324.0,-133677.53465,-0.013394,2017-11-06,2017-11-07,1 days
2,-7689,34,35,1280.62935,1268.0,97107.07215,0.009862,2017-11-08,2017-11-09,1 days
3,-7616,36,37,1305.50560,1312.0,-49461.35040,-0.004975,2017-11-10,2017-11-13,3 days
4,-7404,38,39,1336.35215,1353.0,-123260.68140,-0.012458,2017-11-14,2017-11-15,1 days
...,...,...,...,...,...,...,...,...,...,...
508,-1605,1215,1216,345.28235,347.0,-2756.82825,-0.004975,2022-08-22,2022-08-23,1 days
509,-1583,1217,1218,348.26750,351.0,-4325.54750,-0.007846,2022-08-24,2022-08-25,1 days
510,-1598,1219,1220,342.29720,363.0,-33083.07440,-0.060482,2022-08-26,2022-08-29,3 days
511,-1443,1221,1222,356.22790,362.0,-8329.14030,-0.016203,2022-08-30,2022-08-31,1 days


<Strategy SMAandRSI_WithShortPosition(SMA_short=10,SMA_long=20,upper_bound=50,lower_bound=10,rsi_window=28)>

In [ ]:
df_returns_sorted = df_returns[["_strategy", "Start", "End", "Duration", "Exposure Time [%]", "Equity Final [$]", "Equity Peak [$]", "Return [%]", "Buy & Hold Return [%]", "Return (Ann.) [%]", "Volatility (Ann.) [%]", "Sharpe Ratio", "Sortino Ratio", "Calmar Ratio", "Max. Drawdown [%]", "Avg. Drawdown [%]", "Max. Drawdown Duration", "Avg. Drawdown Duration", "# Trades", "Win Rate [%]", "Best Trade [%]", "Worst Trade [%]", "Avg. Trade [%]", "Max. Trade Duration", "Avg. Trade Duration", "Profit Factor", "Expectancy [%]", "SQN", 
]].sort_values(by = "Equity Final [$]", ascending = False)
df_returns_sorted.to_csv("CombinationStrategies.csv")